In [34]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
import tensorflow as tf
import re
import string
from sklearn.model_selection import train_test_split
from imblearn.under_sampling import RandomUnderSampler
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression
from imblearn.over_sampling import SMOTE
from sklearn.metrics import classification_report
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SpatialDropout1D, Bidirectional, LSTM, GlobalMaxPool1D, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.metrics import classification_report, roc_auc_score
import pickle, os

In [35]:
pip install tensorflow[and-cuda]

  Using cached nvidia_cublas_cu12-12.9.1.4-py3-none-win_amd64.whl.metadata (1.7 kB)
  Using cached nvidia_cuda_cupti_cu12-12.9.79-py3-none-win_amd64.whl.metadata (1.8 kB)
  Using cached nvidia_cuda_nvcc_cu12-12.9.86-py3-none-win_amd64.whl.metadata (1.7 kB)
  Using cached nvidia_cuda_nvrtc_cu12-12.9.86-py3-none-win_amd64.whl.metadata (1.7 kB)
  Using cached nvidia_cuda_runtime_cu12-12.9.79-py3-none-win_amd64.whl.metadata (1.7 kB)
  Using cached nvidia_cudnn_cu12-9.19.0.56-py3-none-win_amd64.whl.metadata (1.9 kB)
  Using cached nvidia_cufft_cu12-11.4.1.4-py3-none-win_amd64.whl.metadata (1.8 kB)
  Using cached nvidia_curand_cu12-10.3.10.19-py3-none-win_amd64.whl.metadata (1.7 kB)
  Using cached nvidia_cusolver_cu12-11.7.5.82-py3-none-win_amd64.whl.metadata (1.9 kB)
  Using cached nvidia_cusparse_cu12-12.5.10.65-py3-none-win_amd64.whl.metadata (1.8 kB)
INFO: pip is looking at multiple versions of tensorflow[and-cuda] to determine which version is compatible with other requirements. This co

ERROR: Cannot install tensorflow[and-cuda]==2.16.1, tensorflow[and-cuda]==2.16.2, tensorflow[and-cuda]==2.17.0, tensorflow[and-cuda]==2.17.1, tensorflow[and-cuda]==2.18.0, tensorflow[and-cuda]==2.18.1, tensorflow[and-cuda]==2.19.0, tensorflow[and-cuda]==2.19.1 and tensorflow[and-cuda]==2.20.0 because these package versions have conflicting dependencies.
ERROR: ResolutionImpossible: for help visit https://pip.pypa.io/en/latest/topics/dependency-resolution/#dealing-with-dependency-conflicts


In [36]:
df_train = pd.read_csv("train.csv")
df_test = pd.read_csv("test.csv")

In [37]:
df_test

,id,comment_text
0,00001cee341fdb12,"Yo bitch Ja Rule is more succesful then you'll ever be whats up with you and hating you sad mofuckas...i should bitch slap ur pethedic white faces and get you to kiss my ass you guys sicken me. Ja rule is about pride in da music man. dont diss that shit on him. and nothin is wrong bein like tupac he was a brother too...fuckin white boys get things right next time.,"
1,0000247867823ef7,"== From RfC == \n\n The title is fine as it is, IMO."
2,00013b17ad220c46,""" \n\n == Sources == \n\n * Zawe Ashton on Lapland — / """
3,00017563c3f7919a,":If you have a look back at the source, the information I updated was the correct form. I can only guess the source hadn't updated. I shall update the information once again but thank you for your message."
4,00017695ad8997eb,I don't anonymously edit articles at all.
...,...,...
153159,fffcd0960ee309b5,". \n i totally agree, this stuff is nothing but too-long-crap"
153160,fffd7a9a6eb32c16,== Throw from out field to home plate. == \n\n Does it get there faster by throwing to cut off man or direct from out fielder? \n Were the out fielders in the Mickey mantle era have better arms? \n Rich
153161,fffda9e8d6fafa9e,""" \n\n == Okinotorishima categories == \n\n I see your changes and agree this is """"more correct."""" I had gotten confused, but then found this: \n :... while acknowledging Japan's territorial rights to Okinotorishima itself ... \n However, is there a category for \n :... did not acknowledge Japan's claim to an exclusive economic zone (EEZ) stemming from Okinotorishima. \n That is, is there a category for """"disputed EEZ""""s? """
153162,fffe8f1340a79fc2,""" \n\n == """"One of the founding nations of the EU - Germany - has a Law of Return quite similar to Israel's"""" == \n\n This isn't actually true, is it? Germany allows people whose ancestors were citizens of Germany to return, but AFAIK it does not allow the descendants of Anglo-Saxons to """"return"""" to Angeln and Saxony. Israel, by contrast, allows all Jews to """"return"""" to Israel, even if they can't trace a particular ancestral line to anyone who lived in the modern state or even mandate Palestine. — """


In [38]:
df_train

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,"Explanation\nWhy the edits made under my username Hardcore Metallica Fan were reverted? They weren't vandalisms, just closure on some GAs after I voted at New York Dolls FAC. And please don't remove the template from the talk page since I'm retired now.89.205.38.27",0,0,0,0,0,0
1,000103f0d9cfb60f,"D'aww! He matches this background colour I'm seemingly stuck with. Thanks. (talk) 21:51, January 11, 2016 (UTC)",0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It's just that this guy is constantly removing relevant information and talking to me through edits instead of my talk page. He seems to care more about the formatting than the actual info.",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on improvement - I wondered if the section statistics should be later on, or a subsection of """"types of accidents"""" -I think the references may need tidying so that they are all in the exact same format ie date format etc. I can do that later on, if no-one else does first - if you have any preferences for formatting style on references or want to do it yourself please let me know.\n\nThere appears to be a backlog on articles for review so I guess there may be a delay until a reviewer turns up. It's listed in the relevant form eg Wikipedia:Good_article_nominations#Transport """,0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember what page that's on?",0,0,0,0,0,0
...,...,...,...,...,...,...,...,...
159566,ffe987279560d7ff,""":::::And for the second time of asking, when your view completely contradicts the coverage in reliable sources, why should anyone care what you feel? You can't even give a consistent argument - is the opening only supposed to mention significant aspects, or the """"most significant"""" ones? \n\n""",0,0,0,0,0,0
159567,ffea4adeee384e90,You should be ashamed of yourself \n\nThat is a horrible thing you put on my talk page. 128.61.19.93,0,0,0,0,0,0
159568,ffee36eab5c267c9,"Spitzer \n\nUmm, theres no actual article for prostitution ring. - Crunch Captain.",0,0,0,0,0,0
159569,fff125370e4aaaf3,And it looks like it was actually you who put on the speedy to have the first version deleted now that I look at it.,0,0,0,0,0,0


In [39]:

print("\nDuplicates in the data = ", df_train.duplicated().sum())
print("\nData Info :- ","\n",df_train.info())


Duplicates in the data =  0
<class 'pandas.DataFrame'>
RangeIndex: 159571 entries, 0 to 159570
Data columns (total 8 columns):
 #   Column         Non-Null Count   Dtype
---  ------         --------------   -----
 0   id             159571 non-null  str  
 1   comment_text   159571 non-null  str  
 2   toxic          159571 non-null  int64
 3   severe_toxic   159571 non-null  int64
 4   obscene        159571 non-null  int64
 5   threat         159571 non-null  int64
 6   insult         159571 non-null  int64
 7   identity_hate  159571 non-null  int64
dtypes: int64(6), str(2)
memory usage: 72.2 MB

Data Info :-  
 None


In [40]:

def clean_text(text):
   
    # lowercase
    text = text.lower()

    # remove Wikipedia markup
    text = re.sub(r"={2,}", " ", text)
    text = re.sub(r":{2,}", " ", text)
    # remove escaped newlines
    text = re.sub(r"\\n|\\t|\\r", " ", text)
    text = re.sub(r"\n|\t|\r", " ", text)

    # remove URLs
    text = re.sub(r"http\S+|www\S+|https\S+", "", text)

    # remove HTML tags
    text = re.sub(r"<.*?>", "", text)

    # remove punctuation
    text = text.translate(str.maketrans("", "", string.punctuation))

    # remove numbers
    text = re.sub(r"\d+", "", text)

    # remove extra spaces
    text = re.sub(r"\s+", " ", text).strip()

    return text


In [41]:
df_train['clean_comment'] = df_train['comment_text'].apply(clean_text)

In [42]:
pd.set_option("display.max_colwidth", None)

In [43]:
df_train.describe()

,toxic,severe_toxic,obscene,threat,insult,identity_hate
count,159571.000000,159571.000000,159571.000000,159571.000000,159571.000000,159571.000000
mean,0.095844,0.009996,0.052948,0.002996,0.049364,0.008805
std,0.294379,0.099477,0.223931,0.054650,0.216627,0.093420
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [44]:
# this shows how many 1's are there in the whole column
display(df_train[['toxic','severe_toxic','obscene','threat','insult','identity_hate']].sum())
display(len(df_train))
   

toxic            15294
severe_toxic      1595
obscene           8449
threat             478
insult            7877
identity_hate     1405
dtype: int64

159571

In [45]:
cmt_labels = ['toxic','severe_toxic','obscene','threat','insult','identity_hate']

freq = df_train[cmt_labels].sum().sort_values(ascending=False)
ratio = freq / len(df_train)

imbalance_df = pd.DataFrame({
    "count": freq,
    "ratio": ratio
})
print(imbalance_df)

               count     ratio
toxic          15294  0.095844
obscene         8449  0.052948
insult          7877  0.049364
severe_toxic    1595  0.009996
identity_hate   1405  0.008805
threat           478  0.002996


In [46]:
# after running the above code we found that only small percentage of the data has toxic values, severe_toxic, threat etc. 
# So most comments are non toxic, so lazy model will detect that every comment is clean and still look accurate.
# as we can see the the data is very imbalanced and the data skewed 

In [47]:
# If we balance the data first then and split the data later copies of the same comment can land on both training and test. 
# The model then sees the twin of the test example during training then the model overfits(Data leakage)

#        Remembering
# original data
# → split
# → balance training only
# → evaluate on untouched test

In [48]:

x = df_train['clean_comment']
y = df_train[cmt_labels]

X_train, X_test, y_train, y_test = train_test_split(
    x,y,
    test_size=0.2, 
    random_state=42,
)

In [49]:
y_train_combo = y_train.astype(str).agg('_'.join, axis=1)
y_train_combo

140030    1_0_0_0_0_0
159124    0_0_0_0_0_0
60006     0_0_0_0_0_0
65432     0_0_0_0_0_0
154979    0_0_0_0_0_0
             ...     
119879    0_0_0_0_0_0
103694    0_0_0_0_0_0
131932    1_0_0_0_0_0
146867    0_0_0_0_0_0
121958    0_0_0_0_0_0
Length: 127656, dtype: str

In [50]:

counts = y_train_combo.value_counts()

majority_class = counts.idxmax()

sampling_strategy = {
    majority_class: 15000   # choose a reasonable cap
}

rus = RandomUnderSampler(
    sampling_strategy=sampling_strategy,
    random_state=42
)

X_under, y_under_combo = rus.fit_resample(
    X_train.to_frame(),
    y_train_combo
)


In [51]:
y_under = y_under_combo.str.split('_', expand=True).astype(int)
y_under.columns = y_train.columns

In [52]:
y_under.sum()

toxic            12238
severe_toxic      1274
obscene           6734
threat             404
insult            6263
identity_hate     1111
dtype: int64

In [53]:
y_train_combo.value_counts().head()

0_0_0_0_0_0    114675
1_0_0_0_0_0      4589
1_0_1_0_1_0      3023
1_0_1_0_0_0      1399
1_0_0_0_1_0       953
Name: count, dtype: int64

In [54]:
# training the model with RUS


vectoriser = TfidfVectorizer(max_features=50000, ngram_range=(1,2))
X_train_vector = vectoriser.fit_transform(X_under['clean_comment'])
X_test_vector = vectoriser.transform(X_test)



In [55]:
y_combo = y_under.astype(str).agg('_'.join, axis=1)


In [56]:

# Apply SMOTE using 'toxic' as the target (binary, both classes have thousands of samples)
smote = SMOTE(k_neighbors=3, random_state=42)

X_smote, y_smote_toxic = smote.fit_resample(X_train_vector, y_under['toxic'])

print("Before SMOTE:", X_train_vector.shape)
print("After SMOTE:", X_smote.shape)
print("\nToxic class distribution after SMOTE:")
print(y_smote_toxic.value_counts())

Before SMOTE: (27981, 50000)
After SMOTE: (31486, 50000)

Toxic class distribution after SMOTE:
toxic
0    15743
1    15743
Name: count, dtype: int64


In [57]:


# Train a OneVsRest Logistic Regression on SMOTE-balanced data
# Note: y_under has all 6 labels aligned with X_train_vector rows
model = OneVsRestClassifier(LogisticRegression(max_iter=1000, random_state=42))
model.fit(X_smote, y_smote_toxic)

print("Model trained successfully!")

Model trained successfully!


In [58]:


# Train on the original undersampled data (all 6 labels)
model = OneVsRestClassifier(
    LogisticRegression(max_iter=1000, class_weight='balanced', random_state=42)
)
model.fit(X_train_vector, y_under)

print("Model trained on all 6 labels!")

Model trained on all 6 labels!


In [59]:

y_pred = model.predict(X_test_vector)

print(classification_report(y_test, y_pred, target_names=cmt_labels))

               precision    recall  f1-score   support

        toxic       0.53      0.86      0.66      3056
 severe_toxic       0.33      0.71      0.45       321
      obscene       0.75      0.80      0.77      1715
       threat       0.29      0.73      0.41        74
       insult       0.60      0.81      0.69      1614
identity_hate       0.32      0.66      0.43       294

    micro avg       0.55      0.82      0.66      7074
    macro avg       0.47      0.76      0.57      7074
 weighted avg       0.58      0.82      0.67      7074
  samples avg       0.07      0.08      0.07      7074



c:\Users\FAROOQUE\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1706: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\FAROOQUE\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1706: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\FAROOQUE\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1706: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no true nor predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])


LSTM 

In [60]:
MAX_WORDS = 50000
MAX_LEN = 200

tokenizer = Tokenizer(num_words=MAX_WORDS, oov_token='<OOV>')
tokenizer.fit_on_texts(X_train)

X_train_seq = pad_sequences(tokenizer.texts_to_sequences(X_train), maxlen=MAX_LEN, padding='post')
X_test_seq = pad_sequences(tokenizer.texts_to_sequences(X_test), maxlen=MAX_LEN, padding='post')

print("Train shape:", X_train_seq.shape)
print("Test shape:", X_test_seq.shape)

Train shape: (127656, 200)
Test shape: (31915, 200)


In [61]:
model = Sequential([
    Embedding(MAX_WORDS, 128, input_length=MAX_LEN),
    SpatialDropout1D(0.3),
    Bidirectional(LSTM(64, return_sequences=True)),
    GlobalMaxPool1D(),
    Dense(64, activation='relu'),
    Dropout(0.3),
    Dense(6, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

c:\Users\FAROOQUE\anaconda3\Lib\site-packages\keras\src\layers\core\embedding.py:100: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ spatial_dropout1d_1             │ ?                      │             0 │
│ (SpatialDropout1D)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_1 (Bidirectional) │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_max_pooling1d_1          │ ?                      │             0 │
│ (GlobalMaxPooling1D)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [62]:
callbacks = [
    EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True, verbose=1),
    ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, min_lr=1e-6, verbose=1)
]

history = model.fit(
    X_train_seq, y_train.values,
    validation_split=0.1,
    batch_size=256,
    epochs=10,
    callbacks=callbacks,
    verbose=1
)

Epoch 1/10
449/449 ━━━━━━━━━━━━━━━━━━━━ 246s 542ms/step - accuracy: 0.7590 - loss: 0.1048 - val_accuracy: 0.9940 - val_loss: 0.0546 - learning_rate: 0.0010
Epoch 2/10
449/449 ━━━━━━━━━━━━━━━━━━━━ 244s 544ms/step - accuracy: 0.9701 - loss: 0.0497 - val_accuracy: 0.9940 - val_loss: 0.0512 - learning_rate: 0.0010
Epoch 3/10
449/449 ━━━━━━━━━━━━━━━━━━━━ 246s 548ms/step - accuracy: 0.9870 - loss: 0.0439 - val_accuracy: 0.9940 - val_loss: 0.0513 - learning_rate: 0.0010
Epoch 4/10
449/449 ━━━━━━━━━━━━━━━━━━━━ 0s 539ms/step - accuracy: 0.9809 - loss: 0.0404
Epoch 4: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
449/449 ━━━━━━━━━━━━━━━━━━━━ 248s 553ms/step - accuracy: 0.9804 - loss: 0.0403 - val_accuracy: 0.9940 - val_loss: 0.0534 - learning_rate: 0.0010
Epoch 5/10
449/449 ━━━━━━━━━━━━━━━━━━━━ 247s 550ms/step - accuracy: 0.9711 - loss: 0.0365 - val_accuracy: 0.9940 - val_loss: 0.0532 - learning_rate: 5.0000e-04
Epoch 5: early stopping
Restoring model weights from the end of

In [63]:
y_pred_proba = model.predict(X_test_seq)
y_pred = (y_pred_proba >= 0.5).astype(int)

print("=== Classification Report ===\n")
print(classification_report(y_test, y_pred, target_names=cmt_labels))

print("\n=== AUC-ROC Scores ===\n")
for i, label in enumerate(cmt_labels):
    auc = roc_auc_score(y_test[label], y_pred_proba[:, i])
    print(f"  {label:15s} AUC-ROC: {auc:.4f}")

overall_auc = roc_auc_score(y_test, y_pred_proba, average='macro')
print(f"\n  Overall (macro) AUC-ROC: {overall_auc:.4f}")

998/998 ━━━━━━━━━━━━━━━━━━━━ 21s 21ms/step
=== Classification Report ===

               precision    recall  f1-score   support

        toxic       0.88      0.73      0.80      3056
 severe_toxic       0.72      0.07      0.12       321
      obscene       0.83      0.75      0.79      1715
       threat       0.00      0.00      0.00        74
       insult       0.73      0.68      0.71      1614
identity_hate       0.00      0.00      0.00       294

    micro avg       0.83      0.66      0.73      7074
    macro avg       0.53      0.37      0.40      7074
 weighted avg       0.78      0.66      0.70      7074
  samples avg       0.06      0.06      0.06      7074


=== AUC-ROC Scores ===

  toxic           AUC-ROC: 0.9748
  severe_toxic    AUC-ROC: 0.9874
  obscene         AUC-ROC: 0.9894
  threat          AUC-ROC: 0.9560
  insult          AUC-ROC: 0.9824
  identity_hate   AUC-ROC: 0.9577

  Overall (macro) AUC-ROC: 0.9746


c:\Users\FAROOQUE\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1706: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\FAROOQUE\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1706: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\FAROOQUE\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1706: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\FAROOQUE\anaconda3

In [64]:
import json

metrics = {
    "classification_report": classification_report(y_test, y_pred, target_names=cmt_labels, output_dict=True),
    "auc_roc_scores": {label: round(roc_auc_score(y_test[label], y_pred_proba[:, i]), 4) for i, label in enumerate(cmt_labels)},
    "training_history": {
        "loss": [float(x) for x in history.history['loss']],
        "val_loss": [float(x) for x in history.history['val_loss']],
        "accuracy": [float(x) for x in history.history['accuracy']],
        "val_accuracy": [float(x) for x in history.history['val_accuracy']],
    }
}
with open("model/metrics.json", "w") as f:
    json.dump(metrics, f, indent=2)
print("✅ Metrics saved!")

✅ Metrics saved!


c:\Users\FAROOQUE\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1706: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\FAROOQUE\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1706: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\FAROOQUE\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1706: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\FAROOQUE\anaconda3

In [65]:

os.makedirs("model", exist_ok=True)
model.save("model/toxicity_model.keras")

with open("model/tokenizer.pickle", "wb") as f:
    pickle.dump(tokenizer, f)

print("✅ Model and tokenizer saved to model/ directory")

✅ Model and tokenizer saved to model/ directory
